<a href="https://colab.research.google.com/github/anshupandey/Generative-AI-opensource/blob/main/TAI_LLM_Function_Calling_with_Mistral-7B_OpenAI_SDK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Integrating tools/function with LLM using Mistral

### Why function calling?

When working with generative text models, it can be difficult to coerce generative models to give consistent outputs in a structured format such as JSON. Function Calling in Gemini allows you to overcome this limitation by forcing the model to output structured data in the format and schema that you define.

You can think of Function Calling as a way to get structured output from user prompts and function definitions, use that structured output to make an API request to an external system, then return the function response to the generative model so that it can generate a natural language summary. In other words, function calling in Gemini helps you go from unstructured text in prompt, to a structured data object, and back to natural language again.

### Benefits of Function Calling

- **Native framework**: Function Calling is a native framework in Gemini, so there's no need to enable additional APIs or install extra packages.
- **Time savings**: No need to write custom code to call, parse, and synthesize information from multiple APIs.
- **Simplified interaction with generative AI models**: Gemini handles the complex task of understanding the user's intent, predicting function calls, extracting relevant function parameters, and generating natural language summaries.
- **Versatility**: Easily extend capabilities by adding more function calls for different APIs and tailoring it to your needs.


In [ ]:
!pip install openai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.8/328.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00


In [ ]:
import os
import json
import requests
import openai

os.environ['TOGETHER_API_KEY'] = '4faeef5f54789f54e13a6e05a6e413be38ba3b30c2059c6a8c9b955a413ce99b'

In [ ]:
client = openai.OpenAI(
  api_key=os.environ.get("TOGETHER_API_KEY"),
  base_url="https://api.together.xyz/v1",
)


model_name="mistralai/Mixtral-8x7B-Instruct-v0.1"
response = client.chat.completions.create(
  model=model_name,
  messages=[
    {"role": "system", "content": "You are a travel agent. Be descriptive and helpful."},
    {"role": "user", "content": "Tell me about San Francisco"},
  ]
)

print(response.choices[0].message.content)

 Absolutely, I'd be happy to tell you about San Francisco! San Francisco is a vibrant and culturally rich city located on the west coast of the United States, in the state of California. It's known for its steep hills, cool summer fog, and iconic landmarks such as the Golden Gate Bridge, Alcatraz Island, and the cable cars.

San Francisco is a city that's full of character and diversity, with a wide range of neighborhoods each with their own unique charm. Some of the most popular areas include Fisherman's Wharf, North Beach, Chinatown, Haight-Ashbury, and the Mission District.

Fisherman's Wharf is a bustling waterfront area that's home to seafood restaurants, shops, and attractions such as the sea lion colony at Pier 39 and the Maritime National Historical Park. North Beach is San Francisco's Little Italy, with plenty of Italian restaurants, cafes, and bakeries. Chinatown is one of the oldest and largest Chinatowns in the country, with colorful streets, markets, and dim sum restaurant

In an API call, you can describe functions and have the model intelligently choose to output a JSON object containing arguments to call one or many functions. The Chat Completions API does not call the function; instead, the model generates JSON that you can use to call the function in your code.

## Define the external function to be integrated

In [ ]:
# Define the OpenWeatherMap API key
OWM_API_KEY = "29af1cea50a401d8e624eea4660b3f59"

def get_current_weather(location, unit="kelvin"):
    """
    Fetches the current weather information for a given location.

    Parameters:
    - location: str, the name of the location (e.g., "Paris").
    - unit: str, the unit of temperature (default is "kelvin").

    Returns:
    - str: JSON formatted string containing weather information.
    """
    # Construct the API request URL
    url = f"https://api.openweathermap.org/data/2.5/weather?q={location}&appid={OWM_API_KEY}"

    # Send the API request
    response = requests.get(url)

    # Parse the temperature and weather forecast from the response
    temp = response.json()['main']['temp']
    forecast = [response.json()['weather'][0]['main'], response.json()['weather'][0]['description']]

    # Create a dictionary with the weather information
    weather_info = {
        "location": location,
        "temperature": temp,
        "unit": 'Kelvin',
        "forecast": forecast
    }

    # Return the weather information as a JSON string
    return json.dumps(weather_info)

In [ ]:
get_current_weather("Paris")

'{"location": "Paris", "temperature": 296.31, "unit": "Kelvin", "forecast": ["Clouds", "broken clouds"]}'

In [ ]:
get_current_weather("Manila")

'{"location": "Manila", "temperature": 300.25, "unit": "Kelvin", "forecast": ["Rain", "heavy intensity rain"]}'

In [ ]:
messages = [{"role":"user",'content':"what is AI?"}]
results = client.chat.completions.create(model = model_name, messages = messages)
print(results.model_dump_json(indent=2))

{
  "id": "8a6aca814d4f6057-ORD",
  "choices": [
    {
      "finish_reason": "eos",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": " AI, or Artificial Intelligence, refers to the development of computer systems or machines that can perform tasks that would normally require human intelligence to accomplish. These tasks can include things like learning, reasoning, problem-solving, perception, and language understanding.\n\nThere are different types of AI, ranging from narrow or weak AI, which is designed to perform a specific task, such as voice recognition or image analysis, to general or strong AI, which has the ability to understand, learn, and apply knowledge across a wide range of tasks at a level equal to or beyond that of a human being.\n\nAI has the potential to transform many industries and aspects of our lives, from healthcare and transportation to education and entertainment. However, it also raises important ethical and social questions relat

In [ ]:

# Define tools/functions for the LLM to use
tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit","kelvin"]},
                    },
                    "required": ["location"],
                },
            },
        },

    ]

In [ ]:
def get_response(messages, tools, model=model_name):
    """
    Handles interaction with the LLM, including making function calls if needed.

    Parameters:
    - messages: list, the conversation history.
    - tools: list, the functions/tools available to the LLM.
    - model: str, the model name to use (default is model_name).

    Returns:
    - str: The response from the LLM or function.
    """
    # Create a chat completion with the given messages and tools
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools,
        tool_choice='auto',
        temperature=0.2,
    )
    #print(response.model_dump_json(indent=2))
    # Get the generated response and tool calls (if any)
    response = response.choices[0].message
    tool_calls = response.tool_calls

    try:
        if tool_calls:
            print("Making a function call")
            # Available functions
            available_functions = {
                "get_current_weather": get_current_weather,
            }
            print(tool_calls)

            # Extend conversation with assistant's reply
            messages.append(response)

            # Handle each function call
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_to_call = available_functions[function_name]
                function_args = json.loads(tool_call.function.arguments)
                function_response = function_to_call(**function_args)
                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_response,
                    }
                )

            # Get a new response from the model with the function response
            second_response = client.chat.completions.create(
                model=model,
                messages=messages,
            )
            return second_response
        else:
            return response.content
    except Exception as e:
        print("Error occurred", e)
        return response

In [ ]:
# Example usage of get_response function
messages = [{"role": "user", "content": "Provide a 2 line explanation for AI"}]
response = get_response(messages, tools)
print(response)

 AI, or Artificial Intelligence, refers to the development of computer systems that can perform tasks that typically require human intelligence. This includes tasks such as learning, problem-solving, decision-making, and understanding natural language.


In [ ]:
messages = [{"role": "user", "content": "How is the weather in Tokyo today?"}]
response = get_response(messages, tools)
print(response.choices[0].message.content)

Making a function call
[ChatCompletionMessageToolCall(id='call_md194vfg929glspn145htd7c', function=Function(arguments='{"location":"Tokyo"}', name='get_current_weather'), type='function')]
 The current temperature in Tokyo is 303.4 Kelvin, which is equivalent to 30.2 degrees Celsius or 86.4 degrees Fahrenheit. The forecast shows broken clouds. Please note that weather conditions can change rapidly, so it's always a good idea to check the most recent weather updates.
